In [ ]:
# Visulizing images side by side.
# Image names and labels are shown.
# Additiionally prediction is made with using the model weights

import torch
import numpy as np
import matplotlib.pyplot as plt

def random_evaluate(model, device, datasetLoader):
    model.eval()
    criterion = nn.BCELoss()

    with torch.no_grad():
        images_1, images_2, targets, path1, path2 = next(iter(datasetLoader))
        img1, img2, target = images_1[0], images_2[0], targets[0]
        output = model(images_1, images_2).squeeze()
        pred = torch.where(output > 0.5, 1, 0)  # get the index of the max log-probability

        img1 = img1.squeeze()
        img1 = img1.movedim(0, -1)
        img2 = img2.squeeze()
        img2 = img2.movedim(0, -1)
        img1 = torch.cat((img1, img2), 1)

        label = target
        plt.imshow(img1)
        plt.show()
        first_image_label = path1[0].split("/")[6]
        second_image_label = path2[0].split("/")[6]

    print(f"1st image: {first_image_label} -- 2nd image: {second_image_label}")
    print(f"Label of the data: {label} (1: Same, 0: Not)")
    print(f"Output: {output[0]}, Prediction: {pred[0]}")

    return output, pred 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load("siamese_network_resnet_A-S-A-S.pt", map_location = device))

random_evaluate(model, device, test_loader)




In [ ]:
# Visulize the ROC curve
# needed package torchmetrics
# pip3 install torchmetrics
import numpy as np
import matplotlib.pyplot as plt
from torchmetrics.classification import BinaryStatScores

def ROCcurve(model, device, test_loader, threshold):
    model.eval()
    correct = 0
    predicted_true = 0

    criterion = nn.BCELoss()

    stat_scores = []
    stat_numpy = []
    metric = BinaryStatScores()

    with torch.no_grad():
        for (images_1, images_2, targets, path1, path2) in test_loader:
            images_1, images_2, targets = images_1.to(device), images_2.to(device), targets.to(device, torch.float32)
            outputs = model(images_1, images_2).squeeze()
            pred = torch.where(outputs > threshold, 1, 0)  # get the index of the max log-probability            
            stat_scores.append(np.array(metric(pred, targets)))

            # Stat_score returns to: TP, FP, TN, FN, Support

    stat_scores = np.array(stat_scores)
    stat_scores = np.asarray(stat_scores)
    scores = stat_scores.sum(axis=0)
    scores = scores / 1000

    # Stat_score returns to: TP, FP, TN, FN, Support
    return scores

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load("siamese_network_resnet_A-S-A-S.pt", map_location = device))

prediction_list = []

for threshold in np.linspace(0,1,5):
    scores = ROCcurve(model, device, test_loader, threshold)
    prediction_list.append(scores)

prediction_list = np.asarray(prediction_list)
true_pos, false_pos, *_ = prediction_list.T

plt.figure()
plt.plot(false_pos, true_pos, 'b')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()



In [ ]:
# Print the number of parameters of the network
def count_parameters(model): 
    print(f"Number of parameters of the network: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = SiameseNetwork().to(device)
count_parameters(model)


